In [34]:
%matplotlib inline
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
import netCDF4
import numpy as np
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import datetime

import createFolder as cF


In [1]:
def create_netCDF_Haukeli(year,month,day,stn_lat,stn_lon,var_name,time_shape):
    thredds  = 'http://thredds.met.no/thredds/dodsC/meps25epsarchive'
    time = '00'
    fn = netCDF4.Dataset('%s/%s/%s/%s/meps_full_2_5km_%s%s%sT%sZ.nc' %(thredds,year,month,day,year,month,day,time),'r')


## Latitudes
## [y = 949][x = 739]
    latitude = fn.variables['latitude']

## Longitudes 
## [y = 949][x = 739]
    longitude = fn.variables['longitude']

## Time
    tid = fn.variables['time']

### From the with Vertical Levels ( height0 ) : 0.0 m

## 1. air_temperature_0m = Surface temperature (T0M)
## [time = 56][height0 = 1][ensemble_member = 10][y = 949][x = 739]
#var_name = 'air_temperature_0m'
    temp_sfc = fn.variables[var_name]    # [K]
    

# Stations latitude longitude, the nearest gridpoint we are looking for
#stn_lat = 59.8
#stn_lon = 7.2

# Now find the absolute value of the difference between the  station's lat/lon with every point in the grid. 
# This tells us how close a point is to the particular latitude and longitude.

    abslat = np.abs(latitude[:,:]-stn_lat)
    abslon= np.abs(longitude[:,:]-stn_lon)

# Now we need to combine these two results. We will use numpy.maximum, which takes two arrays and finds the local 
# maximum.
    c = np.maximum(abslon, abslat)

# If you don't like flattened arrays, you can also get the row/column index like this
    y, x = np.where(c == np.min(c))

### fill values of missing data
    if np.ma.is_masked(temp_sfc[:,:,:,y[0],x[0]]):
        mask = temp_sfc[:,:,:,y[0],x[0]].mask
        fill_value = temp_sfc[:,:,:,y[0],x[0]].fill_value
        marr = np.ma.array(temp_sfc[:,:,:,y[0],x[0]], mask = mask, fill_value = fill_value)
        dt_var = marr.filled().dtype
    else: 
        marr = temp_sfc[:,:,:,y[0],x[0]]
        dt_var = marr.dtype
    
    dt_tid = tid.dtype
    

    
    
### write netCDF file 
    f_sfc = netCDF4.Dataset('../../dataMEPS/%s/%s%s%s_%s.nc' %(var_name,year,month,day,time), 'w')
    f_sfc.createDimension('time', time_shape)            # [time = 67]
    
    if var_name == 'air_pressure_at_sea_level':
        f_sfc.createDimension('height_above_msl', 1) # [height_above_msl = 1]
    else:
        f_sfc.createDimension('height0', 1)          # [height0 = 1]
    
    f_sfc.createDimension('ensemble_member', 10)  # [ensemble_member = 10]
    f_sfc.createDimension('y', 1)                # [y = 1]
    f_sfc.createDimension('x', 1)                # [x = 1]
    
    t = f_sfc.createVariable('time', dt_tid, ('time'),zlib = True)
    if var_name == 'air_pressure_at_sea_level':
        grid_temp_sfc = f_sfc.createVariable('%s' %(var_name), dt_var, ('time','height_above_msl','ensemble_member','y','x'),fill_value = fill_value,zlib=True)
    else:
        grid_temp_sfc = f_sfc.createVariable('%s' %(var_name), dt_var, ('time','height0','ensemble_member','y','x'),fill_value = fill_value,zlib=True)


    t[:] = tid[:]
    grid_temp_sfc[:] = marr.filled()

    f_sfc.close()

In [36]:
# Stations latitude longitude, the nearest gridpoint we are looking for
stn_lat = 59.8
stn_lon = 7.2

year = '2016'
month = '12'

# Variable
var_name = [ \
    ### From the with Vertical Levels ( height0 ) : 0.0 m
#            'air_temperature_0m',                           # [K]
#            'graupelfall_amount',                           # [kg/m^2]
#            'liquid_water_content_of_surface_snow',         # [kg/m^2]   Snow Water Equivivalent (SWE) 
            'precipitation_amount_acc'#,                     # [kg/m^2] 
  #          'rainfall_amount',                              # [kg/m^2]
    #        'snowfall_amount',                              # [kg/m^2]
    ### with Vertical Levels ( height_above_msl ) : 0.0 m
    #        'air_pressure_at_sea_level',                    # [Pa]  
           ]                         

# 16th doesnt exist, the other days have different shapes
t = [
#    1, 2, 3, 4, 5, 6 , 7, 8, 9,10, \
 #   11,12,13,14,15,    17,18,19,20, \
  #  21,22,
     23#,24,25,26,27,28,29,30, \
   # 31
]




for name in var_name:
    cF.createFolder('../dataMEPS/%s/' %(name))
    # Date
    for day in t:
        if day < 10:
            day = '0%s' %(str(day))
        if (day == 17):
            time_shape = 60
        elif (day == 18):
            time_shape = 56
        elif (day == 19):
            time_shape = 53
        else:
            time_shape = 67
        day = str(day)
        create_netCDF_Haukeli(year,month,day,stn_lat,stn_lon,name,time_shape)
        print('File written: %s %s%s%s.nc' %(name,year,month,day))

[[[0.0 -- -- 0.0 0.0 -- -- 0.0 0.0 0.0]]

 [[0.13474273681640625 -- -- -- -- -- -- -- -- --]]

 [[0.9558830261230469 -- -- -- -- -- -- -- -- --]]

 [[1.9340839385986328 -- -- 4.1755523681640625 2.0321998596191406 -- --
   1.2197284698486328 2.5441627502441406 1.7343502044677734]]

 [[2.126209259033203 -- -- -- -- -- -- -- -- --]]

 [[2.4379234313964844 -- -- -- -- -- -- -- -- --]]

 [[2.657196044921875 -- -- 5.963703155517578 10.582990646362305 -- --
   3.6582069396972656 3.0032691955566406 3.3300399780273438]]

 [[2.938518524169922 -- -- -- -- -- -- -- -- --]]

 [[3.2722854614257812 -- -- -- -- -- -- -- -- --]]

 [[3.3156890869140625 -- -- 6.724109649658203 13.572425842285156 -- --
   5.206844329833984 3.876270294189453 4.871124267578125]]

 [[3.4289398193359375 -- -- -- -- -- -- -- -- --]]

 [[3.4646339416503906 -- -- -- -- -- -- -- -- --]]

 [[3.6711349487304688 -- -- 7.3325653076171875 13.66983413696289 -- --
   5.508670806884766 4.139930725097656 5.073947906494141]]

 [[3.68733596